# FAF.FIA16.1 -- Artificial Intelligence Fundamentals

> **Lab 2:** Flocking Behaviour \\
> **Performed by:** Cambur Dumitru, group FAF-191 \\
> **Verified by:** Mihail Gavrilita





## Imports and Utils

In [3]:
import simplequi as simplegui
import math
import random
from utils import Boid


## Task 1 -- Implement the Vector class in Python that works on simple Python lists. The Vector class should implement the vector operations:

In [4]:
class Vector:
    def __init__(self, arr=None):
        if arr is None:
            arr = []
        self.vec = arr
    
    # vector norm/magnitude/distance from origin
    def norm(self):
        return math.sqrt(sum(x ** 2 for x in self.vec))

    # vector addition
    def __add__(self, other):
        return Vector([x + y for x, y in zip(self.vec, other.vec)])
    
    # vector substraction
    def __sub__(self, other):
        return Vector([x - y for x, y in zip(self.vec, other.vec)])
    
    # vector dot product and scalar multiplication
    def __mul__(self, other):
        if isinstance(other, Vector):
            if len(self.vec) != len(other.vec):
                raise ArithmeticError("Vectors must have the same length")
            return sum([x * y for x, y in zip(self.vec, other.vec)])

        if isinstance(other, int) or isinstance(other, float):
            return Vector([x * other for x in self.vec])
        raise TypeError(
            "Multiplication only supported with scalar or vector types")
    
    # scalar division
    def __truediv__(self, other):
        return Vector([x / other for x in self.vec])

    def __str__(self):
        return str(self.vec)
    
    def __repr__(self):
        return str(self.vec)
    
    # isn't a part of the task, however is needed after to find neighbour boids for radius
    def __abs__(self):
        return Vector([abs(self.vec[0]), abs(self.vec[1])])
    
    # implements cross operation on 2d vectors
    def cross_2d(self, other):
        return self.vec[0] * other.vec[1] - self.vec[1] * other.vec[0]

# test on operators
vec = Vector([2, 4])
vec2 = Vector([3, 5])
sum_vec = vec + vec2 # Vector(2+3, 4+5) = Vector(5, 9) -- add
sub_vec = vec - vec2 # vector(2-3, 4-5) = Vector(-1, -1) -- subs
mul_scal = vec * 3 # Vector(2*3, 4*3) = Vector(6, 12) -- scalar mul
div_scal = vec / 0.5 # Vector(2/0.5, 4/0.5) = Vector(4, 8) -- scalar div
dot = vec * vec2 # (2*3 + 4*5) = 26 -- dot prod
cross = vec.cross_2d(vec2) # (2*5 - 4*3) = -2 -- 2d cross prod
print(sum_vec, sub_vec, mul_scal, div_scal, dot, cross)

[5, 9] [-1, -1] [6, 12] [4.0, 8.0] 26 -2


## Task 2 -- Using the Vector class and the provided paper, implement the Boid class with the steering behaviors:

In [5]:
class Boid:
    def __init__(self, sprite):
        self.sprite = sprite
        self.position = Vector(sprite.pos)
        self.velocity = Vector(sprite.vel)
        self.radius = 100
        self.alignment_factor = 3.0
        self.separation_factor = 1.0
        self.cohesion_factor = 2.0

    def get_proximity(self, other_boid):
        return abs(self.position - other_boid.position)

    def alignment(self, sprite_group):
        align_vec = Vector([0, 0])
        proximity_len = 0
        for sprite in sprite_group:
            if self.sprite == sprite:
                continue
            other_boid = Boid(sprite)
            proximity = self.get_proximity(other_boid)
            if proximity.vec[0] <= self.radius and proximity.vec[
                1] <= self.radius:
                proximity_len += 1
                # sum of velocities
                align_vec += other_boid.velocity

        if proximity_len > 0:
            # average velocity
            align_vec /= proximity_len
            align_vec /= align_vec.norm()
            self.velocity += align_vec * self.alignment_factor
            self.velocity /= self.velocity.norm()

    def separation(self, sprite_group):
        separation_vector = Vector([0, 0])
        proximity_len = 0
        for sprite in sprite_group:
            if sprite == self.sprite:
                continue

            other_boid = Boid(sprite)

            proximity = self.get_proximity(other_boid)

            if proximity.vec[0] <= self.radius and proximity.vec[
                1] <= self.radius:
                # calculate the separation vector
                separation_vector += (self.position - other_boid.position)
                proximity_len += 1

        if proximity_len > 0:
            # average the separation vector and apply it to the velocity
            separation_vector /= proximity_len
            self.velocity += separation_vector * self.separation_factor
            self.velocity /= self.velocity.norm()

    def cohesion(self, sprite_group):
        cohesion_vec = Vector([0, 0])
        proximity_len = 0
        for sprite in sprite_group:

            if self.sprite == sprite:
                continue

            other_boid = Boid(sprite)
            proximity = self.get_proximity(other_boid)

            if proximity.vec[0] <= self.radius and proximity.vec[
                1] <= self.radius:
                proximity_len += 1
                # sum of positions
                cohesion_vec += other_boid.position

        if proximity_len > 0:
            cohesion_vec /= proximity_len
            cohesion_vec -= self.position
            self.velocity += cohesion_vec * self.cohesion_factor
            self.velocity /= self.velocity.norm()


As code above is quite hard to illustrate, I will explain it logically. First of all, boids implement three behaviours:

-Alignment
    
    - Makes boids in some local area (represented by radius) to align by the average of their velocity, as every velocity represents the movement by x and y. Giving a boid the average of velocities of nearby boids will make it align with the said boids. This should be used on all boids in some radius.
    
    - The formula can be represented as: alignment = sum(neighbour_velocity1...neighbour_velocityN) / len(neighour_boids)
    
    - We can also multiply it by some factor, which will represent alignment strength
    
    - Vector can be normalized (divided by magnitude) to make velocity uniform

-Cohesion
    
    - Makes boids in some local area to stick to the center of mass. Cohesion vector can be represented as the average between the positions of all neighbours of the said boid. Cohesion vector should be added to current velocity and normalized for boid to actually travel to the position
    - The formula can be represented as: cohesion = sum(neighbour_position1...neighbour_positionN) / len(neighbour_boids)

-Separation

    - Makes boids to separate from each other after reaching some radius respectively to each other. Is represented as the average of differences between current boid position and nearby boid positions. In simple terms, every difference will create a repulsion for the current boid in respect to the nearby boid. By averaging all differences, we will find the averaged direction, where our boid should move to separate from the nearby boids.
    - The formula can be represented as: separation = sum(current_pos-nearby_pos1...current_pos-nearby_posN) / len(neighbour_boids)
    

## Task 3 -- Add the calm flocking behaviour to the Boid class according to the provided paper, using the 3 steering behaviours implemented in the Task 2.

In [6]:
    # flocking is a sum of all three behaviours,
    # i simply call every of these as each function directly modifies the velocity, 
    # therefore sum persists
    def flocking(self, sprite_group):
        self.cohesion(sprite_group)
        self.separation(sprite_group)
        self.alignment(sprite_group)


## Task 4 --  Combine the Boid class with the behaviours implemented in previous tasks with the provided code for the simulation of S. tuberosum and run it in CodeSkulptor. The rocks should exhibit flocking behaviour as implemented in the Boid class.

In [8]:
def boid_handler():
    for rock in rock_group:
        boid = Boid(rock)
        boid.flocking(rock_group)

        rock.pos = boid.position.vec
        rock.vel = boid.velocity.vec

# creates a handler which triggers every 10ms, which goes through each rock,
# converts it into boid and adjust its depending on the neighbours 
# which are found by comparing to every rock from rock_group after converting them into a boid
timer_boid = simplegui.create_timer(10.0, boid_handler)
# -> timer.start() is used in the place game starts

## Conclusions:

After implementing this laboratory work I've about different steering behaviours and in the context of this laboratory work I have learnt to implement these behaviours both separately and as one (flocking). I also changed the example code to change the behaviour of asteroids (rocks) to show how my algorithm works

## Bibliography:

https://www.red3d.com/cwr/boids/

https://www.youtube.com/watch?v=mhjuuHl6qHM